In [1]:
!pip install selenium
!pip install beautifulsoup4
!pip install lxml

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [153]:
import re
import requests
import time

from math import ceil
from selenium import webdriver
from bs4 import BeautifulSoup


class naver_review:
    def __init__(self,product, url):
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-gpu')
        options.add_argument('--no-sandbox') # 크롬 페이지 열리지 않고 구현하도록
        options.add_argument('--disable-dev-shm-usage')
        self.__product = product
        self.__driver = webdriver.Chrome("/usr/local/bin/chromedriver", options=options)
        self.__url = url
        self.__reviews = []

    def get_last_page(self):
        result = requests.get(self.__url)
        soup = BeautifulSoup(result.text, 'html.parser')
        ul_pagination = soup.find("ul", {"class": "filter_top_list__3rOdK"})
        # (2,000)으로 출력됨
        before_parse_str = ul_pagination.find(
            "li", {"class": "filter_on__X0_Fb"}).find("em").text
        # 정규 표현식을 통해 ()와 , 을 제거
        total_number_of_reviews = int(re.sub(r'\D', '', before_parse_str))
        # 전체 페이지 개수를 구함
        total_page = ceil(total_number_of_reviews / 20)

        return total_page

    def get_reviews(self, num_of_total_page):
        driver = self.__driver
        product = self.__product
        driver.get(self.__url)
        driver.set_window_size(1920, 1080)

        print(f"{product} 크롤링 시작 -----------------------------------------")
        print(f"전체 페이지 : {num_of_total_page}")
        # 전체 페이지 개수만큼 크롤링
        for page in range(num_of_total_page):
            self.change_page(page+1)

            time.sleep(0.5)
            now_page_before_parse = driver.find_element_by_css_selector(
                '[class="reviewItems_list_review__1sgcJ"] + .pagination_pagination__2M9a4 [class="pagination_now__gZWGP"').text
            num_of_now_page = int(
                (now_page_before_parse.replace("현재 페이지", "")).strip())  

            print(f"현재 크롤링 중인 페이지 : {num_of_now_page}")
            self.extract_review()

        self.__driver.quit()
        print("End Scraping")
        return self.__reviews

    def change_page(self, num_of_next_page):
        driver = self.__driver
        # 페이지 변경
        driver.find_element_by_css_selector(
            f"[class='reviewItems_list_review__1sgcJ'] + .pagination_pagination__2M9a4 > a[data-nclick='N=a:rev.page,r:{num_of_next_page}']"
        ).click()

    def extract_review(self):
        # 페이지에서 리뷰데이터만 추출
        driver = self.__driver
        num_of_reviews = len(driver.find_elements_by_css_selector(
            '[class="reviewItems_list_review__1sgcJ"] > li'))

        # 현재 페이지의 리뷰 개수 (20개) 만큼 반복
        for index in range(num_of_reviews):
            # 평점 크롤링
            rating_before_parse = driver.find_element_by_css_selector(
                f".reviewItems_list_review__1sgcJ > li:nth-child({index + 1}) .reviewItems_average__16Ya-").text
            rating = (rating_before_parse.replace("평점", "")).strip()

            # 날짜 크롤링
            date = ""
            before_date_list = driver.find_elements_by_css_selector(
                f".reviewItems_list_review__1sgcJ > li:nth-child({index + 1}) .reviewItems_etc__1YqVF")
            # date 값 전처리 (날짜만 뽑아내도록)
            for date_list in before_date_list:
                if "." in date_list.text:
                    date = date_list.text
                    break

            # 제목 크롤링
            title = driver.find_element_by_css_selector(
                f".reviewItems_list_review__1sgcJ > li:nth-child({index + 1}) .reviewItems_review_text__2Bwpa > em").text
            # 내용 크롤링
            content = (driver.find_element_by_css_selector(
                f".reviewItems_list_review__1sgcJ > li:nth-child({index + 1}) .reviewItems_review_text__2Bwpa > p").text).strip()

            self.__reviews.append({
                '평점': rating,
                '날짜': date,
                '제목': title,
                '내용': content
            })

In [154]:
import argparse
import sys
import csv


def save_to_csv(name, reviews):
    print("Start Save --- " , name)
    file_url = name+"_review.csv"
    file = open(file_url, "w", encoding="utf-8-sig")
    writer = csv.writer(file)
    writer.writerow(["평점", "날짜", "제목", "내용"])
    for review in reviews:
        writer.writerow(list(review.values()))
    return True


if __name__=='__main__':
    Product_Name = ["헤라블랙쿠션", "닥터지마일드선크림", "삼성갤럭시버즈"]
    URL_List = ["https://search.shopping.naver.com/catalog/26539564522?cat_id=50005288&nv_mid=26539564522","https://search.shopping.naver.com/catalog/11415677047?cat_id=50000445&nv_mid=11415677047", "https://search.shopping.naver.com/catalog/23702664490?cat_id=50002334&nv_mid=23702664490" ]

    for i in range(len(URL_List)):
        NAVER_SHOP_URL = URL_List[i]
        naver = naver_review(Product_Name[i],NAVER_SHOP_URL)

        '''현재 네이버에서 제공해주는 개수 2000개라 최대 100페이지라 100으로 설정'''
        last_page = 100 

        reviews = naver.get_reviews(last_page)

        save_to_csv(Product_Name[i], reviews)

헤라블랙쿠션 크롤링 시작 -----------------------------------------
전체 페이지 : 100
현재 크롤링 중인 페이지 : 1
현재 크롤링 중인 페이지 : 2
현재 크롤링 중인 페이지 : 3
현재 크롤링 중인 페이지 : 4
현재 크롤링 중인 페이지 : 5
현재 크롤링 중인 페이지 : 6
현재 크롤링 중인 페이지 : 7
현재 크롤링 중인 페이지 : 8
현재 크롤링 중인 페이지 : 9
현재 크롤링 중인 페이지 : 10
현재 크롤링 중인 페이지 : 11
현재 크롤링 중인 페이지 : 12
현재 크롤링 중인 페이지 : 13
현재 크롤링 중인 페이지 : 14
현재 크롤링 중인 페이지 : 15
현재 크롤링 중인 페이지 : 16
현재 크롤링 중인 페이지 : 17
현재 크롤링 중인 페이지 : 18
현재 크롤링 중인 페이지 : 19
현재 크롤링 중인 페이지 : 20
현재 크롤링 중인 페이지 : 21
현재 크롤링 중인 페이지 : 22
현재 크롤링 중인 페이지 : 23
현재 크롤링 중인 페이지 : 24
현재 크롤링 중인 페이지 : 25
현재 크롤링 중인 페이지 : 26
현재 크롤링 중인 페이지 : 27
현재 크롤링 중인 페이지 : 28
현재 크롤링 중인 페이지 : 29
현재 크롤링 중인 페이지 : 30
현재 크롤링 중인 페이지 : 31
현재 크롤링 중인 페이지 : 31
현재 크롤링 중인 페이지 : 33
현재 크롤링 중인 페이지 : 34
현재 크롤링 중인 페이지 : 35
현재 크롤링 중인 페이지 : 36
현재 크롤링 중인 페이지 : 37
현재 크롤링 중인 페이지 : 38
현재 크롤링 중인 페이지 : 39
현재 크롤링 중인 페이지 : 40
현재 크롤링 중인 페이지 : 41
현재 크롤링 중인 페이지 : 42
현재 크롤링 중인 페이지 : 43
현재 크롤링 중인 페이지 : 44
현재 크롤링 중인 페이지 : 45
현재 크롤링 중인 페이지 : 46
현재 크롤링 중인 페이지 : 47
현재 크롤링 중인 페이지 : 48
현재 크롤링 중인 페이지 : 49
현재 크롤링 중인

## 